In [1]:
import pandas as pd
import altair as alt

In [2]:
from datetime import datetime, timedelta
 
def descobre_mes(ano, num_semana):    
    date = str(ano) + '-W' + str(num_semana)
    res = datetime.strptime(date + '-1', "%Y-W%W-%w")
    return res.strftime("%B")

In [43]:
def extrair_elementos(lista_str):
    try:
        lista = ast.literal_eval(lista_str)
        return ', '.join(lista)
    except (SyntaxError, ValueError):
        return None

In [44]:
def remover_duplicatas(lista_str):
    try:
        lista = list(dict.fromkeys(lista_str))
        return lista
    except (SyntaxError, ValueError):
        return None

In [18]:
import ast

hit_songs = pd.read_csv('../songs_data/hit_songs/spotify_hits_dataset_complete.csv', delimiter='\t')
hit_songs = hit_songs.explode(['artist_id','artist_name']).reset_index(drop=True)
hit_songs['artist_id'] = hit_songs['artist_id'].apply(extrair_elementos)

artists = pd.read_csv('../songs_data/artists/spotify_artists_info_complete_reduced_genres.csv', delimiter='\t')
total_charts = pd.read_csv('../my_data/charts/total_charts.csv')

songs_and_artists = pd.merge(hit_songs, artists, on='artist_id')
charts = pd.merge(total_charts, songs_and_artists, on='song_id')


0                            6M25WWmNO7wK1DnKmd3z6R
1    3dKjmeRZcSUw617CoMzEA0, 2jku7tDXc6XoB6MO2hFuqg
2                            74XFHRwlV6OrjEM0A2NCMF
3                            4Kxlr1PRlDKEB0ekOCyHgX
4                            31lnFZEM6ysvjOx59VyxRE
Name: artist_id, dtype: object
0    1WgXqy2Dd70QQOU7Ay074N
1    2dXKh2aHr7RS2RVPlWJoUS
2    293zczrfYafIItmnmM3coR
3    4DqRDyrUuJO8CuxfgVCU3m
4    1U5zgr455OGyIkLNXvDdrf
Name: artist_id, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158596 entries, 0 to 158595
Data columns (total 40 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   position               158596 non-null  int64  
 1   song_id                158596 non-null  object 
 2   song_name_x            158557 non-null  object 
 3   artist                 158559 non-null  object 
 4   streams                158596 non-null  int64  
 5   last_week_position     137014 non-null  float64
 6 

In [19]:
# ajustando japao
charts.loc[(charts['country'] == 'japan') & (charts['year'] == 2017), 'week'] += 34

In [20]:
charts[(charts['country'] == 'japan') & (charts['year']==2017)]['week'].describe()

count    2453.000000
mean       43.642886
std         5.183625
min        35.000000
25%        39.000000
50%        44.000000
75%        48.000000
max        52.000000
Name: week, dtype: float64

In [21]:
charts['month'] = charts.apply(lambda row: descobre_mes(row['year'], row['week']), axis=1)

In [24]:
charts.columns

Index(['position', 'song_id', 'song_name_x', 'artist', 'streams',
       'last_week_position', 'weeks_on_chart', 'peak_position',
       'position_status', 'year', 'week', 'country', 'song_name_y',
       'artist_id', 'artist_name', 'popularity_x', 'explicit', 'song_type',
       'track_number', 'num_artists', 'num_available_markets', 'release_date',
       'duration_ms', 'key', 'mode', 'time_signature', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness',
       'speechiness', 'valence', 'tempo', 'name', 'followers', 'popularity_y',
       'genres', 'image_url', 'month'],
      dtype='object')

In [26]:
colunas_de_interesse = ['artist_id', 'artist_name', 'year', 'month', 'streams', 'song_name_x', 'genres', 'country', 'image_url']
data = charts[colunas_de_interesse]

In [47]:
agg_func = {
    'artist_name': 'first', 
    'streams': 'sum', 
    'song_name_x': list, 
    'genres': list,  
    'image_url': 'first'
}
data_analysis = data.groupby(['artist_id', 'year', 'month', 'country']).agg(agg_func).reset_index()
data_analysis['genres'] = data_analysis['genres'].apply(lambda x: list(pd.Series(x).explode().unique()))
data_analysis['artist_name'] = data_analysis['artist_name'].apply(extrair_elementos)
data_analysis['song_name_x'] = data_analysis['song_name_x'].apply(remover_duplicatas)
data_analysis.head()

,artist_id,year,month,country,artist_name,streams,song_name_x,genres,image_url
0,00FQb4jTyendYWaN8pK0wa,2017,April,united_kingdom,Lana Del Rey,1147562,[Love],[['pop']],https://i.scdn.co/image/6e8dc460cfb6b89d797030...
1,00FQb4jTyendYWaN8pK0wa,2017,February,australia,Lana Del Rey,356843,[Love],[['pop']],https://i.scdn.co/image/6e8dc460cfb6b89d797030...
2,00FQb4jTyendYWaN8pK0wa,2017,February,brazil,Lana Del Rey,1108298,[Love],[['pop']],https://i.scdn.co/image/6e8dc460cfb6b89d797030...
3,00FQb4jTyendYWaN8pK0wa,2017,February,canada,Lana Del Rey,546300,[Love],[['pop']],https://i.scdn.co/image/6e8dc460cfb6b89d797030...
4,00FQb4jTyendYWaN8pK0wa,2017,February,france,Lana Del Rey,160031,[Love],[['pop']],https://i.scdn.co/image/6e8dc460cfb6b89d797030...


<h1>Gráfico</h1>

In [ ]:
dropdown_year = alt.binding_select(
    options=data_reduced['year'].unique().tolist(),
    name='Escolha o ano '
)
year_param = alt.param(
    value=2017,
    bind=dropdown_year
)

dropdown_country = alt.binding_select(
    options=['australia', 'brazil', 'canada', 'france', 'germany', 'global', 'japan', 'united_kingdom', 'united_states'],
    name='Escolha um país ou visão global '
)
ycol_param = alt.param(
    value='global',
    bind=dropdown_country
)


alt.Chart(data_analysis).mark_image(
    width=50,
    height=50
).encode(
    x='x',
    y='y',
    url='img'
)